# 1. 패키지 불러오기

In [1]:
import requests   # http/https 통신 
from bs4 import BeautifulSoup  # parsing
import ujson # 데이터파싱 (저장용)

import time # delay

# 2. 카테고리 저장

In [2]:
# https://sports.news.naver.com/kbaseball/news/index?date=20231208&isphoto=N&type=team&team=LG
# LG/ 2023-12-08/

# https://sports.news.naver.com/kbaseball/news/index?date=20231207&isphoto=N&type=team&team=KT
# KT/ 2023-12-07/

#date와 team 두개를 찾아야됨

In [3]:
req = requests.get("https://sports.news.naver.com/kbaseball/news/index?date=20231208&isphoto=N&type=team&team=LG")
html = req.text
soup = BeautifulSoup(html, 'lxml')
soup_category = soup.select("div.news_team ul li")   

CATEGORY = dict()
for s in soup_category:
    CATEGORY.update({s.get_text(strip=True) : s["data-id"]})

In [4]:
print(CATEGORY)

{'전체': 'kbo', 'LG': 'LG', 'KT': 'KT', 'SSG': 'SK', 'NC': 'NC', '두산': 'OB', 'KIA': 'HT', '롯데': 'LT', '삼성': 'SS', '한화': 'HH', '키움': 'WO'}


# 3. 셀레니움 설치하기

## 3-1. 셀레니움 설치

In [5]:
!pip install selenium
!pip install webdriver_manager

In [6]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## 3-2. 크롬드라이버 버전 확인

In [7]:
# Chrome 드라이버 생성
driver = webdriver.Chrome()

# 버전 정보 가져오기
print("Chrome 드라이버 버전:", driver.capabilities['chrome']['chromedriverVersion'])

# 드라이버 종료
driver.quit()

Chrome 드라이버 버전: 122.0.6261.111 (9d4c1072da62b411b351a38b9ed6214ab236aa7b-refs/branch-heads/6261@{#1015})


# 4. 웹 크롤링 진행

## 4-1. 아티클 뽑아오기

In [8]:
URL_TMPL = "https://sports.news.naver.com/kbaseball/news/index?date={}&isphoto=N&type=team&team={}"

article_info = []

driver = webdriver.Chrome()

with open("crawling_results/ilovebaseball.jsonl", "w", encoding="UTF-8") as output_file:
    for cname, cid in CATEGORY.items():
        if cname == '전체': #중복을 피하기 위해 전체 카테고리는 넘어가기
            continue
        print("수집중인 카테고리: [{}]".format(cname))


        for i in range(10):
            date = str(20230630-i) #23년 6월 30일부터 9일 전까지 수집


            driver.get(url=URL_TMPL.format(date,cid))
            
            content = driver.find_elements(By.CSS_SELECTOR, "li div.text a.title") #제목, url이 들어있음
            des = driver.find_elements(By.CSS_SELECTOR, "li div.text p.desc")  #뉴스 내용이 들어있음
            comp = driver.find_elements(By.CSS_SELECTOR, "li div.text div.source span.press")  #뉴스 회사 이름이 들어있음

            for e, d, c in zip(content, des, comp):
                title = e.text
                url = e.get_attribute("href")
                description = d.text
                company = c.text

                data = {"title": title, "url": url, "cname": cname, "description": description, "company": company}

                article_info.append(data)
                print(ujson.dumps(data, ensure_ascii=False), file=output_file)
    
            
            page_block = driver.find_elements(By.CSS_SELECTOR, "div.paginate") #다음 페이지로 넘어가기
            for i in page_block:
                if i.value_of_css_property("display") == "block":
                    abc = i.text
                    for i in range(len(abc)):
                        if i%2 != 0 or i == 0:
                            continue
                        else:
                            page=f"&page={abc[i]}"

                            driver.get(url=URL_TMPL.format(date,cid)+page)

                            content = driver.find_elements(By.CSS_SELECTOR, "li div.text a.title") #제목, url이 들어있음
                            des = driver.find_elements(By.CSS_SELECTOR, "li div.text p.desc")  #뉴스 내용이 들어있음
                            comp = driver.find_elements(By.CSS_SELECTOR, "li div.text div.source span.press")  #뉴스 회사 이름이 들어있음

                            for e, d, c in zip(content, des, comp):
                                title = e.text
                                url = e.get_attribute("href")
                                description = d.text
                                company = c.text

                                data = {"title": title, "url": url, "cname": cname, "description": description, "company": company}

                                article_info.append(data)
                                print(ujson.dumps(data, ensure_ascii=False), file=output_file) #반복됨..
                     

                elif i.value_of_css_property("display") == "none":
                    continue
                else:
                    print(f"예상치 못한 문제가 {i.value_of_css_property('display')} 라는 이름으로 나타났습니다.")
                time.sleep(0.5)


            
        time.sleep(0.5)

수집중인 카테고리: [LG]
수집중인 카테고리: [KT]
수집중인 카테고리: [SSG]
수집중인 카테고리: [NC]
수집중인 카테고리: [두산]
수집중인 카테고리: [KIA]
수집중인 카테고리: [롯데]
수집중인 카테고리: [삼성]
수집중인 카테고리: [한화]
수집중인 카테고리: [키움]


In [9]:
print(article_info) #제대로 크롤링 됨

[{'title': "'김현수 끝내기' LG, 승차 마진 +20...롯데는 3연승 반등", 'url': 'https://sports.news.naver.com/kbaseball/news/read?oid=241&aid=0003285272', 'cname': 'LG', 'description': "김현수 '1점 추가 오케이!' (서울=연합뉴스) 임화영 기자 = 19일 오후 서울 잠실야구장에서 열린 프로야구 한화 이글스와 LG 트윈스의 경기. 5회말 무...", 'company': '일간스포츠'}, {'title': '\'3점차 뒤집고 끝내기\' 염경엽 감독 "팀이 조금씩 더 강해지는 느낌을 받는다" [잠실 톡톡]', 'url': 'https://sports.news.naver.com/kbaseball/news/read?oid=109&aid=0004880502', 'cname': 'LG', 'description': '[OSEN=잠실, 김성락 기자] LG가 KIA에 짜릿한 끝내기 승리를 거뒀다.LG는 30일 서울 잠실구장에서 열린 2023시즌 KBO리그 KIA와 경기에서 ...', 'company': 'OSEN'}, {'title': '‘완벽한 6월 마무리’ LG 염경엽 감독 “팀이 조금씩 더욱 강해지는 느낌을 받는다”[SS잠실in]', 'url': 'https://sports.news.naver.com/kbaseball/news/read?oid=468&aid=0000957436', 'cname': 'LG', 'description': 'LG 염경엽 감독이 5월 30일 잠실 롯데전에서 승리한 후 선수들과 하이파이브 하고 있다. 서울 | 연합뉴스[스포츠서울 | 잠실=윤세호기자] LG 염경엽 감...', 'company': '스포츠서울'}, {'title': '\'승패마진 +20\' 염경엽 감독 "팀이 조금씩 더 강해지는 느낌 받는다"', 'url': 'https://sports.news.naver.com/kbaseball/news/read?oid=311&aid=000

In [10]:
#전체 아티클 개수

print(len(article_info)) #2851

2851


In [11]:
#한화 아티클 개수

all=0

for a in article_info:
    if a['cname'] == '한화':
        all+=1
print(all)

427


## 4-2. 겹친 기사의 위치를 출력하는 함수 생성

In [12]:
def remove_duplicate_keys(my_dict, keys_to_check):
    seen_values = {key: set() for key in keys_to_check} 
    #ex. seen_values == {'title':{'a','b'...}, 'description':{'abc','def'...}}
    duplicated_data = [] #겹친 데이터의 위치를 저장

    for key in keys_to_check:
        for idx, item in enumerate(my_dict):
            value = item[key]
            if value in seen_values[key]:
                j_set = set()
                j_set.add(idx) #idx는 데이터의 위치를 제공함
                for a, it in enumerate(my_dict):
                    if it[key] == value:
                        j_set.add(a) #value 값과 겹친 데이터들을 전부 j_set안에 넣음
                if j_set in duplicated_data:
                    pass #그래서 생성된 j_set이 이미 duplicated_data 안에 있는지 확인하는 과정
                else:
                    duplicated_data.append(j_set)
                    
            else:
                seen_values[key].add(value)
    #duplicated_data 안에 겹친 데이터들의 위치를 set이 묶인 리스트로 저장해 놓음
    #ex.[{0, 1}, {19, 100}, {120, 30}, {121, 31}, ... {835, 860, 502}, {864, 429}]
    
    #그러나 이 int데이터들을 전부 꺼내 하나의 리스트 안에 넣어보니 겹친 것들이 있었음 ex.160, 68, 162 etc..

    
    return(duplicated_data)

    
d = remove_duplicate_keys(article_info, ['title','description']) #d 안에 겹친 데이터들끼리의 위치를 반환

In [13]:
print(d)

#[{202, 204}, {1906, 1907}, {72, 89}, {106, 132, 149}, ...] 겹친 데이터들의 위치를 set 안에 저장함

[{202, 204}, {1934, 1935}, {72, 89}, {106, 132, 149}, {145, 155, 167}, {248, 260}, {257, 277}, {280, 273}, {284, 276}, {288, 285, 270}, {322, 316}, {325, 343}, {176, 353}, {361, 279}, {363, 349}, {366, 359}, {256, 387}, {474, 479}, {513, 451, 501, 521}, {722, 724}, {602, 727}, {769, 767}, {771, 751}, {789, 773, 766}, {788, 756}, {810, 819}, {946, 955}, {2704, 986, 975}, {985, 994}, {1009, 1011, 1023}, {1012, 1020, 1006}, {1018, 998}, {1052, 1044}, {973, 2742, 1063}, {706, 1086}, {908, 1092}, {1113, 1110}, {1212, 1150}, {1238, 1215}, {2601, 1331, 1364, 2614, 2585}, {1376, 1217}, {1248, 1443}, {1491, 1469}, {1538, 1498, 1515}, {1544, 1594, 1557}, {1576, 1548}, {1630, 1638}, {1697, 1671}, {1720, 1700}, {1705, 1724}, {1744, 1717}, {473, 1771}, {1859, 1854}, {1675, 1861}, {1908, 1893}, {1578, 1957}, {2026, 2022, 2070}, {2072, 2058}, {2073, 2041}, {2085, 1998}, {2128, 2121, 2007, 2111}, {2115, 2118}, {2144, 2136}, {2312, 2292}, {2354, 2422}, {2577, 2603}, {2612, 2607}, {2658, 1211, 2700}, {1

In [14]:
for i in d:
    k_list = []
    i_list = list(i) #set을 리스트로 바꿔 인덱싱 가능하게 만들음
    for a in range(len(i_list)):
        d_data = {article_info[i_list[a]]["title"] : article_info[i_list[a]]["description"]}
        k_list.append(d_data)
    print(k_list)
    
#제목과 간략 기사 내용 중 겹치는 것들을 뽑아본 결과 전부 같은 기사임. (제목과 기사 내용이 같은 경우 영상 유무에 따라 두개 생성된 같은 기사/ 
#설명만 같고 제목이 다를 경우 같은 기자가 작성한 같은 기사)
#만약 이 둘을 위배하더라도 하나의 기사는 표절로 봐야되기 때문에 같은 기사로 가정했음.

[{'KBO 전반기 최고 빅매치 ‘SSG-LG전’ 대상 프로토 승부식 한경기구매 게임 발매': '26일(월) 오후 2시부터 2023 KBO SSG-LG전 대상 프로토 승부식 74회차 한경기구매 게임 발매KBO 전반기 최고 빅매치, SSG-LG전을 대상으...'}, {'KBO 전반기 최고 빅매치 ‘SSG-LG전’ 대상 프로토 승부식 한경기구매 게임 발매': '26일 오후 2시부터 2023 KBO SSG-LG전 대상 프로토 승부식 74회차 한경기구매 게임 발매선두 자리 놓고 치열한 주중 3연전 펼쳐질 예정…27일 ...'}]
[{'[부고] 김헌곤(삼성 라이온즈 외야수)씨 외조모상': '[스타뉴스 | 안호근 기자] ■ 이현자씨 별세, 김헌곤(삼성 라이온즈 외야수)씨 외조모상, 27일, 빈소 대구보훈병원 장례식장 104호(대구광역시 달서구 월...'}, {'[부고] 김헌곤(삼성 라이온즈 외야수)씨 외조모상': '[마이데일리] 김헌곤(삼성 라이온즈 외야수)씨 외조모상= 망인: 故 이현자, 27일, 빈소: 대구보훈병원 장례식장 104호(대구광역시 달서구 월곡로 60),...'}]
[{'번트 헛스윙→번트 파울→극적 동점 3루타...출루왕의 포효, 이런 모습 처음이야': '[OSEN=인천, 민경훈 기자] 28일 오후 인천 SSG랜더스필드에서 ‘2023 신한은행 SOL KBO리그’ SSG 랜더스와 LG 트윈스의 경기가 열렸다.8...'}, {'‘1:0→1:6→6:6→8:6’ 뒷심의 LG, 또 약속의 8회 역전승, 1위 지켰다…SSG 필승조 무너지다 [인천 리뷰]': '[OSEN=인천, 민경훈 기자] 28일 오후 인천 SSG랜더스필드에서 ‘2023 신한은행 SOL KBO리그’ SSG 랜더스와 LG 트윈스의 경기가 열렸다.8...'}]
[{"'연타석 홈런' 김민성, 왜 라인업 제외됐나...신민재 2루수, 염갈량의 노림수": '[OSEN=인천, 민경훈 기자] 27일 오후 인천 SSG랜더스필드에서 ‘2023 신한은행 SOL KBO리그’ SSG 랜더스와 LG 트윈스의 경기가 열렸다.6

## 4-3. 저장된 위치 데이터에서 두번 겹친 위치가 있는지 확인하기

위에 remove_duplicate_keys 함수는 제목에서 겹친 것들끼리 set로 묶고 <br>
다시 한번 description에서 겹친 것들끼리 set로 묶기 때문에 아래와 같은 상황이 생길 수 있음.

In [15]:
#예시

a= {'title':'HI', 'description': 'ABC'}
b= {'title':'HI', 'description': 'DEF'}
c= {'title':'BYE', 'description': 'ABC'}

ex_list = [a,b,c]

example = remove_duplicate_keys(ex_list, ['title','description'])
print(example) #[{0, 1}, {0, 2}]

[{0, 1}, {0, 2}]


- 이렇게 출력됐을 때 각 set당 하나의 데이터만 살리고 모두 지워버릴 경우<br>
- [{0, 1}, {0, 2}] 에서 1번째, 0번째 데이터가 지워지면 첫번째 set의 데이터가 전부 지워지게 되는 오류가 발생한다.<br>
<br>
- 그러므로 저장된 위치 데이터에서 두번 겹친 위치가 있는지 확인해준다.all_the_list : 다른 기사들과 title이나 description에서 하나라도 겹친 경우가 있는 기사들을 나열한 리스트double_duplicated : title에서와 description에서 각각 다른 기사와 겹친 기사들을 모아놓은 리스트

In [16]:
all_the_list = []
for i in d:
    i_list = list(i)
    for i in range(len(i_list)):
        all_the_list.append(i_list[i])
print(all_the_list)

[202, 204, 1934, 1935, 72, 89, 106, 132, 149, 145, 155, 167, 248, 260, 257, 277, 280, 273, 284, 276, 288, 285, 270, 322, 316, 325, 343, 176, 353, 361, 279, 363, 349, 366, 359, 256, 387, 474, 479, 513, 451, 501, 521, 722, 724, 602, 727, 769, 767, 771, 751, 789, 773, 766, 788, 756, 810, 819, 946, 955, 2704, 986, 975, 985, 994, 1009, 1011, 1023, 1012, 1020, 1006, 1018, 998, 1052, 1044, 973, 2742, 1063, 706, 1086, 908, 1092, 1113, 1110, 1212, 1150, 1238, 1215, 2601, 1331, 1364, 2614, 2585, 1376, 1217, 1248, 1443, 1491, 1469, 1538, 1498, 1515, 1544, 1594, 1557, 1576, 1548, 1630, 1638, 1697, 1671, 1720, 1700, 1705, 1724, 1744, 1717, 473, 1771, 1859, 1854, 1675, 1861, 1908, 1893, 1578, 1957, 2026, 2022, 2070, 2072, 2058, 2073, 2041, 2085, 1998, 2128, 2121, 2007, 2111, 2115, 2118, 2144, 2136, 2312, 2292, 2354, 2422, 2577, 2603, 2612, 2607, 2658, 1211, 2700, 1026, 2708, 2189, 2823]


In [17]:
double_duplicated = []
for i in all_the_list:
    if all_the_list.count(i) > 1:
        double_duplicated.append({i:all_the_list.count(i)})
    else:
        pass

print(double_duplicated)


[]


- 혹시나 두번 겹친게 있나 확인해봤는데 [cid = 전체]아티클을 뺐을 경우엔 두번 겹친게 없었다!
- 그러므로 함수 수정은 안해도 될 듯!!

## 4-4. 중복 데이터 제거

- 밑에서 반응점수를 확인한 후 반응점수가 제일 높은 기사를 살리고 나머지를 삭제하고 싶으므로 밑에서 이어서 제거하겠습니다!
- 일단은 d에 겹친 데이터들의 위치를 저장해놨습니다

# 5. 반응점수

## 5-1. 반응점수 구하기(2차 웹크롤링)

#### 반응점수 구하기
- 좋아요:1점, 슬퍼요:1.5점, 화나요:2점, 팬이에요:2점, 후속기사 원해요:3점/
- (이 기사를 추천합니다 는 추가점수 2점)의 가중치로 점수를 부여한다.
<br>

* *(이 기사를 추천합니다는 한명의 사용자가 위에 다섯개 중 하나와 중복 투표할 수 있음)

In [18]:
driver = webdriver.Chrome()

for i in range(len(article_info)):
    leehansol_choigo = article_info[i]['url']
    driver.get(url=leehansol_choigo)
    
    Interest_Score = 0
    try:
        good = driver.find_elements(By.CSS_SELECTOR, "li.good span._count") #좋아요
        Interest_Score += int(good[1].text)

        sad = driver.find_elements(By.CSS_SELECTOR, "li.sad span._count") #슬퍼요
        Interest_Score += (1.5)*int(sad[1].text)
        angry = driver.find_elements(By.CSS_SELECTOR, "li.angry span._count") #화나요
        Interest_Score += 2*int(angry[1].text)
        fan = driver.find_elements(By.CSS_SELECTOR, "li.fan span._count") #팬이에요
        Interest_Score += 2*int(fan[1].text)
        want = driver.find_elements(By.CSS_SELECTOR, "li.want span._count") #후속기사 원해요
        Interest_Score += 3*int(want[1].text)

        recommendation = driver.find_element(By.CSS_SELECTOR, "em.u_cnt") #이 기사를 추천합니다
        if recommendation.text == "":
            pass
        else:
            Interest_Score += 2*int(recommendation.text)   


        article_info[i].update({'Interest Score':Interest_Score})
    except:
        print(f"{i}번째 아티클에서 오류가 발생했습니다.")
    
    if i % 20 == 0:
        time.sleep(0.5)

297번째 아티클에서 오류가 발생했습니다.
645번째 아티클에서 오류가 발생했습니다.
693번째 아티클에서 오류가 발생했습니다.
944번째 아티클에서 오류가 발생했습니다.
1985번째 아티클에서 오류가 발생했습니다.
1992번째 아티클에서 오류가 발생했습니다.
1995번째 아티클에서 오류가 발생했습니다.
2002번째 아티클에서 오류가 발생했습니다.
2004번째 아티클에서 오류가 발생했습니다.
2015번째 아티클에서 오류가 발생했습니다.
2021번째 아티클에서 오류가 발생했습니다.
2027번째 아티클에서 오류가 발생했습니다.
2028번째 아티클에서 오류가 발생했습니다.
2036번째 아티클에서 오류가 발생했습니다.
2080번째 아티클에서 오류가 발생했습니다.
2103번째 아티클에서 오류가 발생했습니다.
2106번째 아티클에서 오류가 발생했습니다.
2117번째 아티클에서 오류가 발생했습니다.
2336번째 아티클에서 오류가 발생했습니다.
2402번째 아티클에서 오류가 발생했습니다.
2450번째 아티클에서 오류가 발생했습니다.


## 5-2. 오류난 위치에서 다른 방식으로 반응점수 구하기

- 오류난 기사들을 보면 좋아요가 "1892"가 아닌 "1,892"같은 식으로 콤마가 껴있어서 int로 변환이 안돼 오류가 났었다.
- 추가로 오류가 난 위치들에 쉼표 제거 함수를 넣어서 다시 구해보자.

In [62]:
errored_num = [297,645,693,944,1985,1992,1995,2002,2004,2015,2021,2027,2028,2036,2080,2103,2106,2117,2336,2402,2450]

In [63]:
# 쉼표 제거 후 정수로 변환
def remove_commas(value):
    return int(value.replace(',', ''))

In [64]:
driver = webdriver.Chrome()

for i in errored_num:

    try:
        driver.get(url = article_info[i]["url"])

        Interest_Score = 0

        good = driver.find_elements(By.CSS_SELECTOR, "li.good span._count") #좋아요
        Interest_Score += remove_commas(good[1].text)

        sad = driver.find_elements(By.CSS_SELECTOR, "li.sad span._count") #슬퍼요
        Interest_Score += (1.5)*remove_commas(sad[1].text)
        angry = driver.find_elements(By.CSS_SELECTOR, "li.angry span._count") #화나요
        Interest_Score += 2*remove_commas(angry[1].text)
        fan = driver.find_elements(By.CSS_SELECTOR, "li.fan span._count") #팬이에요
        Interest_Score += 2*remove_commas(fan[1].text)
        want = driver.find_elements(By.CSS_SELECTOR, "li.want span._count") #후속기사 원해요
        Interest_Score += 3*remove_commas(want[1].text)

        recommendation = driver.find_element(By.CSS_SELECTOR, "em.u_cnt") #이 기사를 추천합니다
        if recommendation.text == "":
            pass
        else:
            Interest_Score += 2*remove_commas(recommendation.text)   

        article_info[i].update({'Interest Score':Interest_Score})
    except:
        print(f"{i}번째 아티클에서 오류가 발생했습니다.")

In [65]:
for i in errored_num:
    print(article_info[i]) #잘 나온다!

{'title': '동점 만든 주자-야수 충돌…LG 내야수들은 왜 주루방해 판정에 당황했나', 'url': 'https://sports.news.naver.com/kbaseball/news/read?oid=477&aid=0000436688', 'cname': 'LG', 'description': '▲ 오지환 황성빈 ⓒ곽혜미 기자▲ 오지환 ⓒ곽혜미 기자[스포티비뉴스=잠실, 신원철 기자] 보기 드문 플레이 하나가 경기 결과를 바꿨다. LG 트윈스는 23일...', 'company': '스포티비뉴스', 'Interest Score': 758.5}
{'title': '홈런·타점·득점·장타율 4개 부문 1위, 최정 40홈런 페이스…36세 시즌에 첫 MVP 되나', 'url': 'https://sports.news.naver.com/kbaseball/news/read?oid=109&aid=0004876736', 'cname': 'SSG', 'description': '[OSEN=인천, 김성락 기자] 7회말 SSG 선두타자 최정이 다시 앞서가는 솔로 홈런을 날린 뒤 세리머니를 하고 있다. 2023.06.24 /ksl0919...', 'company': 'OSEN', 'Interest Score': 544.0}
{'title': '\'2100G 출장 자축포\' 최정, "타격감 완벽하지 않은데 결과 괜찮아서 다행" [잠실 톡톡]', 'url': 'https://sports.news.naver.com/kbaseball/news/read?oid=109&aid=0004875554', 'cname': 'SSG', 'description': '[OSEN=잠실, 민경훈 기자]7회초 1사 SSG 최정이 우익수 오른쪽 2루타를 날린 후 2루에서 조동화 코치와 하이파이브를 하고 있다. 2023.06.22...', 'company': 'OSEN', 'Interest Score': 172.0}
{'title': '“원준아 너무 미안해, 다음 등판 때는 꼭…” 승리에도 활짝 못 웃은 ‘허 캡틴’'

## 5-3. 반응점수가 높은 순으로 기사 100개 정렬하기

In [66]:
for i in range(len(article_info)):
    try:
        float(article_info[i]['Interest Score'])
    except:
        print(i)
        
# Interest Score 전부 있는지 확인

In [67]:
# 'score' 키의 값으로 정렬
sorted_data = sorted(article_info, key =lambda x: x['Interest Score'], reverse=True)

# 상위 100개 뽑기
top_100_scores = sorted_data[:100]

In [68]:
# 결과 출력
print(top_100_scores)

[{'title': '\'논란 폭발\' 오재원 결국 마이크 내려놓는다 "그동안 감사했습니다"', 'url': 'https://sports.news.naver.com/kbaseball/news/read?oid=108&aid=0003162628', 'cname': '삼성', 'description': '[스타뉴스 | 김우종 기자] 오재원이 26일 개인 SNS(소셜네트워크서비스)를 통해 해설위원을 그만둔다는 글을 올렸다. /사진=오재원 SNS 오재원. /사진...', 'company': '스타뉴스', 'Interest Score': 9871.5}, {'title': '양창섭과 \'빈볼\' 신경전 뒤끝? 오재원 해설위원, 대놓고 "조성훈 승리&SSG 스윕 부탁" 발언 논란', 'url': 'https://sports.news.naver.com/kbaseball/news/read?oid=109&aid=0004877248', 'cname': '삼성', 'description': '오재원 SPOTV 해설위원 / OSEN DB[OSEN=손찬익 기자] 지난 24일 문학 삼성-SSG전에서 양창섭(삼성)의 빈볼을 확신하는 발언으로 논란을 일으...', 'company': 'OSEN', 'Interest Score': 5407.0}, {'title': '"스윕 부탁드린다" 뒤끝 작렬…오재원에게서 찾을 수 없었던 \'학습효과+해설 자질\'', 'url': 'https://sports.news.naver.com/kbaseball/news/read?oid=117&aid=0003743999', 'cname': '삼성', 'description': "[마이데일리 = 박승환 기자] 하루가 멀다 하고 구설수에 오르고 있다. '소신발언'과 생각 없이 내뱉는 말에는 분명 차이가 있다. 스스로 받았던 상처를 고스...", 'company': '마이데일리', 'Interest Score': 5013.5}, {'title': '“해설로 선수 바보 만들어” 직격한 오재원, 비슷한 논란으로 해설위원 사퇴'

In [69]:
ojw_mentioned = 0
for idx, i in enumerate(top_100_scores[:10]): #반응점수 톱10에서 오재원 언급량 찾기
    if "오재원" in i["title"] or "오재원" in i["description"]:
        ojw_mentioned+=1
    else:
        print(f"{idx}에서 오재원 언급 안됨") #몇번째에서 오재원이 언급이 안됐는지 위치 찾기

print(ojw_mentioned) #1~9등 기사가 전부 오재원 기사

9에서 오재원 언급 안됨
9


In [70]:
#탑 10 점수
Top_ten_Score = []
for i in top_100_scores[:10]:
    Top_ten_Score.append(i['Interest Score'])

print(Top_ten_Score)

[9871.5, 5407.0, 5013.5, 3451.0, 3360.0, 3347.5, 3201.5, 3127.0, 3054.5, 2990.0]


- 오재원이 논란이 많네요...;; 참고로 저도 싫어합니다 ㅋ

## 5-4. 이어서 중복 데이터 제거(반응 점수가 높은 기사 하나만 남기기)

- 순서대로 204, 1935 ... 등 위치에 있는 데이터들을 삭제하려고 해도 첫 데이터를 삭제하면 리스트 길이가 1 짧아져 두번째부턴 엉뚱한게 삭제되는 것을 발견했다.
- 그러므로 enumerate를 활용해 index가 맞는 위치만 삭제하도록 한다.

In [71]:
print(d)

[{202, 204}, {1934, 1935}, {72, 89}, {106, 132, 149}, {145, 155, 167}, {248, 260}, {257, 277}, {280, 273}, {284, 276}, {288, 285, 270}, {322, 316}, {325, 343}, {176, 353}, {361, 279}, {363, 349}, {366, 359}, {256, 387}, {474, 479}, {513, 451, 501, 521}, {722, 724}, {602, 727}, {769, 767}, {771, 751}, {789, 773, 766}, {788, 756}, {810, 819}, {946, 955}, {2704, 986, 975}, {985, 994}, {1009, 1011, 1023}, {1012, 1020, 1006}, {1018, 998}, {1052, 1044}, {973, 2742, 1063}, {706, 1086}, {908, 1092}, {1113, 1110}, {1212, 1150}, {1238, 1215}, {2601, 1331, 1364, 2614, 2585}, {1376, 1217}, {1248, 1443}, {1491, 1469}, {1538, 1498, 1515}, {1544, 1594, 1557}, {1576, 1548}, {1630, 1638}, {1697, 1671}, {1720, 1700}, {1705, 1724}, {1744, 1717}, {473, 1771}, {1859, 1854}, {1675, 1861}, {1908, 1893}, {1578, 1957}, {2026, 2022, 2070}, {2072, 2058}, {2073, 2041}, {2085, 1998}, {2128, 2121, 2007, 2111}, {2115, 2118}, {2144, 2136}, {2312, 2292}, {2354, 2422}, {2577, 2603}, {2612, 2607}, {2658, 1211, 2700}, {1

In [72]:
# d == [{202, 204}, {1934, 1935}, {72, 89}, ... ]
d_list = []

for i in d:
    d_l = list(i)
    scores = []
    one = 0
    for k in d_l:
        scores.append(article_info[k]['Interest Score'])
    for k in d_l:    
        if article_info[k]['Interest Score'] != max(scores):
            d_list.append(k)
        else:
            one +=1
    if one ==1:
        pass
    else:
        print(f"{d_l}에서 Interest Score 중복되는게 발견됨")

print(d_list) #잘 뽑힌 것 같음
#[202, 1935, 72, 106, 149, 155,...]

[202, 1935, 72, 106, 149, 155, 167, 248, 257, 280, 284, 285, 270, 316, 325, 176, 279, 363, 359, 256, 474, 513, 501, 521, 724, 602, 769, 751, 789, 773, 756, 810, 946, 2704, 986, 985, 1009, 1011, 1012, 1020, 998, 1044, 973, 1063, 1086, 1092, 1113, 1212, 1215, 2601, 1331, 1364, 2614, 1217, 1443, 1469, 1498, 1515, 1544, 1557, 1548, 1630, 1671, 1700, 1724, 1744, 473, 1854, 1675, 1893, 1957, 2026, 2022, 2072, 2073, 1998, 2121, 2007, 2111, 2118, 2144, 2312, 2354, 2603, 2612, 1211, 2700, 1026, 2189]


In [73]:
a_info = article_info
article_info=[]
deleted_number = 0

for index, element in enumerate(a_info):
    if index not in d_list:
        article_info.append(element)
    else:
        try:
            d_list.remove(index)
            print(f"{index}위치에 있는 데이터 삭제")
            deleted_number +=1
        except:
             print(f"{index}위치에 있는 데이터 삭제 실패")
print(f"총 삭제한 데이터 개수{deleted_number}")                

72위치에 있는 데이터 삭제
106위치에 있는 데이터 삭제
149위치에 있는 데이터 삭제
155위치에 있는 데이터 삭제
167위치에 있는 데이터 삭제
176위치에 있는 데이터 삭제
202위치에 있는 데이터 삭제
248위치에 있는 데이터 삭제
256위치에 있는 데이터 삭제
257위치에 있는 데이터 삭제
270위치에 있는 데이터 삭제
279위치에 있는 데이터 삭제
280위치에 있는 데이터 삭제
284위치에 있는 데이터 삭제
285위치에 있는 데이터 삭제
316위치에 있는 데이터 삭제
325위치에 있는 데이터 삭제
359위치에 있는 데이터 삭제
363위치에 있는 데이터 삭제
473위치에 있는 데이터 삭제
474위치에 있는 데이터 삭제
501위치에 있는 데이터 삭제
513위치에 있는 데이터 삭제
521위치에 있는 데이터 삭제
602위치에 있는 데이터 삭제
724위치에 있는 데이터 삭제
751위치에 있는 데이터 삭제
756위치에 있는 데이터 삭제
769위치에 있는 데이터 삭제
773위치에 있는 데이터 삭제
789위치에 있는 데이터 삭제
810위치에 있는 데이터 삭제
946위치에 있는 데이터 삭제
973위치에 있는 데이터 삭제
985위치에 있는 데이터 삭제
986위치에 있는 데이터 삭제
998위치에 있는 데이터 삭제
1009위치에 있는 데이터 삭제
1011위치에 있는 데이터 삭제
1012위치에 있는 데이터 삭제
1020위치에 있는 데이터 삭제
1026위치에 있는 데이터 삭제
1044위치에 있는 데이터 삭제
1063위치에 있는 데이터 삭제
1086위치에 있는 데이터 삭제
1092위치에 있는 데이터 삭제
1113위치에 있는 데이터 삭제
1211위치에 있는 데이터 삭제
1212위치에 있는 데이터 삭제
1215위치에 있는 데이터 삭제
1217위치에 있는 데이터 삭제
1331위치에 있는 데이터 삭제
1364위치에 있는 데이터 삭제
1443위치에 있는 데이터 삭제
1469위치에 있는 데이터 삭제
1498위치에 있는 데이터 삭제
1515위치에 있는 데이터 삭제
1544위치에 있는 데

In [74]:
print(2851-89)
print(len(article_info))

2762
2762


In [75]:
print(d_list) #삭제할거 더 없음!

[]


# 6. 아티클 리스트에서 필요한 정보 뽑아오기

## 6-1. 관심 있는 선수들 언급량 비교

- 관심 있는 선수인 노시환, 수리, 문동주, 이정후, 오재원(뜨거운 감자)가 몇번 언급됐는지 확인해본다.
- 키워드는 제목과 간략 내용정리에서 찾고, 한 기사 중 제목과 간략 내용 둘 다 있었더라도 하나의 키워드로 평가한다.
<br>
* ***만약 기자 이름과 선수 이름이 동일할 경우 잘못 카운트 될 수 있음

In [76]:
nsh = 0 #노시환
nsh_score = 0 #노시환 스코어

sr = 0 #한화 마스코트 수리
sr_score = 0 #수리 스코어

mdj = 0 #문동주
mdj_score = 0 #문동주 스코어

ljh = 0 #이정후 (kbo의 슈퍼스타)
ljh_score = 0 #이정후 스코어

ojw = 0 #오재원 (뜨거운 감자)
ojw_score = 0 #오재원 스코어


for i in article_info:
    if "노시환" in i["title"] or "노시환" in i["description"]:
        nsh+=1
        nsh_score+=i['Interest Score']
    if "수리" in i["title"] or "수리" in i["description"]:
        sr+=1
        sr_score+=i['Interest Score']
    if "문동주" in i["title"] or "문동주" in i["description"]:
        mdj= mdj+1
        mdj_score+=i['Interest Score']    
    if "이정후" in i["title"] or "이정후" in i["description"]:
        ljh= ljh+1
        ljh_score+=i['Interest Score']
    if "오재원" in i["title"] or "오재원" in i["description"]:
        ojw= ojw+1
        ojw_score+=i['Interest Score']

print("날짜 : 2023-06-21 ~ 2023-06-30")
print("키워드 언급량 / 관심도 점수"),print("----------------------------------------"),print('')
print(f"노시환:{nsh}번 / {nsh_score}점")
print(f"수리:{sr}번 / {sr_score}점")
print(f"문동주:{mdj}번 / {mdj_score}점")
print(f"이정후:{ljh}번 / {ljh_score}점")
print(f"오재원:{ojw}번 / {ojw_score}점")

날짜 : 2023-06-21 ~ 2023-06-30
키워드 언급량 / 관심도 점수
----------------------------------------

노시환:54번 / 12473.5점
수리:11번 / 2258.0점
문동주:60번 / 17894.0점
이정후:27번 / 5085.0점
오재원:58번 / 76466.5점


### 오재원은 언급량이 문동주보다 적지만 반응점수가 4배 이상 크기 때문에,
### 6.21~6.30은 오재원의 시기였다 !!

- 노시환 좀만 더 분발하자!!

## 6-2. 좋아하는 선수의 기사만 뽑아내기

In [77]:
#노시환 리스트

nsh_list = []

for i in article_info:
    if "노시환" in i["title"] or "노시환" in i["description"]:
        nsh_list.append({i["title"]:i["description"]})

print(nsh_list)

[{'\'천군만마\' 토종 에이스 1군 복귀 임박 "나균안 컨디션 100%…4일 한화전 선발 출격" [울산포커스]': '2023 KBO리그 롯데자이언츠와 한화이글스의 경기가 15일 부산사직야구장에서 열렸다. 롯데 나균안이 4회초 1사 1루에서 노시환을 병살로 잡으며 이닝을 마...'}, {'한화, 파죽의 7연승… 무려 18년만': '문동주 6이닝 무실점… 윌리엄스 결승타노시환 2경기 연속 홈런… 15개 단독 2위30일 오후 대구 삼성라이온즈파크에서 열린 2023 프로야구 한화 이글스와 ...'}, {'[IS 포커스] ‘강한’ 1·2번, ‘강타’ 외인까지…이번엔 진짜 ‘다이너마이트’다': "한화 노시환이 28일 대전 KT전에서 홈런을 친 후 후속 타자 닉 윌리엄스와 축하를 나누고 있다. 사진=한화 제공이번엔 진짜로 '다이너마이트' 냄새가 난다...."}, {'“마음 한 켠에 투수의 꿈 남아 있어…” 생애 첫 올스타전 출전 앞둔 노시환의 색다른 소망 [MK인터뷰]': '“투수의 꿈이 마음 한 켠에 자그만하게 남아있다. 올스타전에서 투수로 나가게 된다면 삼자범퇴로 막겠다.”생애 첫 올스타전 출격을 앞둔 노시환(한화 이글스)이...'}, {'한화 윌리엄스 ‘공·수·주’ 맹활약…“다이너마이트 타선 될 것 같다” 노시환도 반색': '지난 28일 KT전에서 2루타를 날린 윌리엄스가 팬들의 함성을 받으며 달리고 있다. 한화 제공한화의 새 외국인 타자 닉 윌리엄스(30·외야수)의 KBO리그 ...'}, {"'만년 꼴찌' 한화..이젠 7연승으로 진기록 도전": '한화 이글스 노시환. 사진=뉴시스 제공한화 이글스가 1371일 만에 6연승을 기록했습니다.한화는 지난 28일 대전 한화생명 이글스파크에서 열린 KT 위즈와의...'}, {'"한화가 왜 9위죠?" 6연승에 뛰어든 복덩이 타자는 이해가 안 된다 [인터뷰]': '▲ 닉 윌리엄스(왼쪽)와 노시환. ⓒ한화 이글스▲ 28일 2회에 나온 윌리엄스의 첫 안타 순간. ⓒ한화 이글스[스포티비뉴스=대전, 고유라 기자] 한화 이글스...'}

In [78]:
print(len(nsh_list))

54


### 네이버스포츠-야구를 웹크롤링하면서...

야구에 워낙 관심이 많아서 야구 현황을 알아보기 위해 6/30 부터 10일 전까지 10개 구단의 기사들을 크롤링해봤습니다.<br>
중복 포함 크롤링된 기사 개수는 2851개이고, 관심 있는 한화와 노시환 선수를 위주로 찾아봤습니다. <br>
<br>
기사를 크롤링하면서 겹친 기사를 삭제하는 시간도 갖고...<br>
각 기사마다 사용자 반응점수를 부과해 인기 많은 기사를 제 방식대로 순서매기기도 했습니다<br>
<br>
예를 들어 언급량은 문동주가 제일 많았지만<br>
반응점수 top 9의 기사들이 전부 오재원의 기사였기 때문에 6.21~6.30은 오재원에게 가장 관심이 쏠렸다고 해석할 수 있습니다<br>
<br>
다음번에 또 크롤링하고 싶은 시기나 선수가 생기면 찾아보고 이에 따라 어떤 상관관계가 있는지 사회분석을 해보도록 하겠습니다<br>
감사합니다!